In [33]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad

In [34]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("VineReview").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()


In [35]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://xeniae-vine.s3.amazonaws.com/vine_table.csv"
spark.sparkContext.addFile(url)
vine_df = spark.read.csv(SparkFiles.get("vine_table.csv"), sep=",", header=True, inferSchema=True)

In [36]:
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RMDCHWD0Y5OZ9|          3|            0|          1|   N|                N|
| RZSL0BALIYUNU|          5|            0|          0|   N|                Y|
| RIZR67JKUDBI0|          3|            0|          1|   N|                Y|
|R27HL570VNL85F|          5|            0|          0|   N|                Y|
|R34EBU9QDWJ1GD|          5|            0|          0|   N|                Y|
|R1WCUI4Z1SIQEO|          5|            0|          0|   N|                N|
| RL5LNO26GAVJ1|          2|            3|          4|   N|                Y|
|R3GYQ5W8JHP8SB|          5|            0|          0|   N|                Y|
|R30SHYQXGG5EYC|          5|            0|          0|   N|                Y|
|R14YLXA56NP51I|          5|            1|          1|   N|     

In [37]:
#Load in a SQL function to use columns
from pyspark.sql.functions import  col
# filtered DF to create a DataFrame with 20 or more total votes
vine_filtered = vine_df.filter(col('total_votes')>=20)
vine_filtered.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2243Y3OD8U6KQ|          5|           47|         61|   N|                N|
|R2TGT0CDTCAAHW|          5|           21|         23|   N|                Y|
| RX4D22YSXEF4P|          1|           37|         38|   N|                Y|
|R3FL2NTLFUSPTQ|          5|           33|         37|   N|                N|
|R3QTP3YNZXAPPF|          3|           23|         24|   N|                Y|
|R36V6V42VN5AS5|          5|           34|         37|   N|                Y|
|R27LZWE27BJPOB|          5|           22|         23|   N|                N|
|  RMRD6SMF2AUQ|          3|            4|         21|   N|                N|
| RMPCXKWX3T57Y|          1|            1|         73|   N|                N|
|R1P7GJ0IN2BRNH|          5|           37|         37|   N|     

In [51]:
# filter DF where percentage of helpful_votes is equal to or greater than 50%
vine_hv_filtered = vine_filtered.filter(col('helpful_votes')/col('total_votes') >= 0.5)
vine_hv_filtered.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2243Y3OD8U6KQ|          5|           47|         61|   N|                N|
|R2TGT0CDTCAAHW|          5|           21|         23|   N|                Y|
| RX4D22YSXEF4P|          1|           37|         38|   N|                Y|
|R3FL2NTLFUSPTQ|          5|           33|         37|   N|                N|
|R3QTP3YNZXAPPF|          3|           23|         24|   N|                Y|
|R36V6V42VN5AS5|          5|           34|         37|   N|                Y|
|R27LZWE27BJPOB|          5|           22|         23|   N|                N|
|R1P7GJ0IN2BRNH|          5|           37|         37|   N|                Y|
|R2R6JPF9KOD2HJ|          5|           19|         20|   N|                Y|
|R2J0ZZGFXKM8KR|          2|           21|         22|   N|     

In [52]:
# filter all Vine review 
vine_filtered_y = vine_hv_filtered.filter(col('vine') == 'Y')
vine_filtered_y.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1R9RU7JW0MFR2|          4|           20|         23|   Y|                N|
|R19EFYNN3W8Q07|          5|           26|         32|   Y|                N|
|R34DJ1R8AEU0SG|          5|           29|         35|   Y|                N|
|R25P5CXK5L9RHF|          5|          146|        161|   Y|                N|
|R2E9VZB3I4LSN5|          5|           55|         59|   Y|                N|
| RKYLHZL7EPELX|          4|           19|         25|   Y|                N|
|R1U13EKGQD3ZE6|          5|           22|         25|   Y|                N|
| RYW05F1MUEF01|          5|           87|        102|   Y|                N|
|R2SW4NXNO7HZJ5|          4|           28|         33|   Y|                N|
|R2016NFLSUR97Y|          2|           26|         37|   Y|     

In [53]:
# filter all NON-Vine review 
vine_filtered_n = vine_hv_filtered.filter(col('vine') == 'N')
vine_filtered_n.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2243Y3OD8U6KQ|          5|           47|         61|   N|                N|
|R2TGT0CDTCAAHW|          5|           21|         23|   N|                Y|
| RX4D22YSXEF4P|          1|           37|         38|   N|                Y|
|R3FL2NTLFUSPTQ|          5|           33|         37|   N|                N|
|R3QTP3YNZXAPPF|          3|           23|         24|   N|                Y|
|R36V6V42VN5AS5|          5|           34|         37|   N|                Y|
|R27LZWE27BJPOB|          5|           22|         23|   N|                N|
|R1P7GJ0IN2BRNH|          5|           37|         37|   N|                Y|
|R2R6JPF9KOD2HJ|          5|           19|         20|   N|                Y|
|R2J0ZZGFXKM8KR|          2|           21|         22|   N|     

In [67]:
# total count of Vine and NON-Vine reviews
count_y_reviews = vine_filtered_y.agg({"review_id":"count"}).collect().pop()['count(review_id)']
count_n_reviews = vine_filtered_n.agg({"review_id":"count"}).collect().pop()['count(review_id)']
print(f"total number of Vine reviews: {count_y_reviews}, total number of NON-Vine reviews: {count_n_reviews}")

total number of paid reviews: 60, total number of unpaid reviews: 14477


In [68]:
# VINE total number of reviews, the number of 5-star reviews, and the percentage 5-star reviews
vine_y = vine_filtered_y.groupby('star_rating').agg({'star_rating':'count'}).withColumnRenamed("count(star_rating)", "ratings_count").sort(col('star_rating').desc())
vine_y_percent = vine_y.withColumn("ratings_percentage", (vine_y.ratings_count/count_y_reviews)*100)
vine_y_percent.show()

+-----------+-------------+------------------+
|star_rating|ratings_count|ratings_percentage|
+-----------+-------------+------------------+
|          5|           34|56.666666666666664|
|          4|           16|26.666666666666668|
|          3|            9|              15.0|
|          2|            1|1.6666666666666667|
+-----------+-------------+------------------+



In [71]:
#NON VINE total number of reviews, the number of 5-star reviews, and the percentage 5-star reviews
vine_n = vine_filtered_n.groupby('star_rating').agg({'star_rating':'count'}).withColumnRenamed("count(star_rating)", "ratings_count").sort(col('star_rating').desc())
vine_n_percent = vine_n.withColumn("ratings_percentage", (vine_n.ratings_count/count_n_reviews)*100)
vine_n_percent.show()

+-----------+-------------+------------------+
|star_rating|ratings_count|ratings_percentage|
+-----------+-------------+------------------+
|          5|         8212| 56.72445948746287|
|          4|         2687|18.560475236582164|
|          3|         1292| 8.924500932513642|
|          2|          754| 5.208261380120191|
|          1|         1532| 10.58230296332113|
+-----------+-------------+------------------+

